In [1]:
from lunax.data_processing.utils import *
from lunax.models import xgb_reg
from lunax.hyper_opt import OptunaTuner

In [2]:
df_train = load_data('./example_data/regression/train.csv', 'csv')
df_train

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031
...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,Learning Lab,Episode 25,75.66,Education,69.36,Saturday,Morning,NaN,0.0,Negative,56.87058
749996,749996,Business Briefs,Episode 21,75.75,Business,35.21,Saturday,Night,NaN,2.0,Neutral,45.46242
749997,749997,Lifestyle Lounge,Episode 51,30.98,Lifestyle,78.58,Thursday,Morning,84.89,0.0,Negative,15.26000
749998,749998,Style Guide,Episode 47,108.98,Lifestyle,45.39,Thursday,Morning,93.27,0.0,Negative,100.72939


In [3]:
target = 'Listening_Time_minutes'

In [4]:
df_train = preprocess_data(df_train, target)

In [5]:
X_train, X_val, y_train, y_val = split_data(df_train, target)

In [6]:


# 创建调参器 使用默认搜索空间
tuner = OptunaTuner(
    n_trials=1
)

# 执行优化
results = tuner.optimize("XGBRegressor", X_train, y_train, X_val, y_val)

# 获取最优参数
best_params = results['best_params']

[I 2025-05-14 21:03:01,958] A new study created in memory with name: no-name-21011836-eec4-4088-b9bf-525a95e4aab0


[lunax]> XGBoost Parameter Explanations:
[Model complexity parameters]>
- lambda: 	L2 regularization. Smoother than L1. Better for sparse data. Prevents overfitting.
- reg_lambda/alpha: 	Regularization. Control model complexity. Prevents overfitting.
- gamma: 	TREE ONLY. Minimum loss reduction for split. Prevents overfitting.
- max_depth: 	Higher = more complex model. Prevents overfitting.
- subsample: 	Number of samples per tree. Prevents overfitting.
- colsample_bytree: 	Fraction of features used per tree. Prevents overfitting.
- min_child_weight: 	Minimum sum of instance weight in a child. Prevents overfitting.


[Training and Optimization Parameters]>
- eta: 	Learning rate.
- booster: 	"gbtree" for nonlinear features. "gblinear" for linear features
- grow_policy: 	Controls how new nodes are added to the tree. "lossguide" for best split. "depthwise" for best depth.




[I 2025-05-14 21:03:04,917] Trial 0 finished with value: 735.8300406602353 and parameters: {'booster': 'gblinear', 'seed': 0, 'eta': 0.11462430255904295, 'lambda': 33.47005382991086, 'reg_lambda': 0.6814853490908365, 'reg_alpha': 146}. Best is trial 0 with value: 735.8300406602353.


In [7]:
xgb_reg = xgb_reg(best_params)
xgb_reg.fit(X=X_train, y=y_train)

[lunax]> model training finished.


In [8]:
xgb_reg.evaluate(X_val, y_val)

[lunax]> target value description:
+-------+--------+--------+-------+----------+
|   min |    max |   mean |   std |   median |
+=======+========+========+=======+==========+
|     0 | 119.97 |   45.4 | 27.13 |    43.36 |
+-------+--------+--------+-------+----------+
[lunax]> model evaluation results:
+-----------+--------+--------+-------+------+
| metrics   |   rmse |    mse |   mae |   r2 |
+===========+========+========+=======+======+
| values    |  27.13 | 735.83 | 22.64 |   -0 |
+-----------+--------+--------+-------+------+


{'rmse': 27.126187359454615,
 'mse': 735.8300406602353,
 'mae': 22.63824864131461,
 'r2': -3.675867769725727e-06}